In [13]:
import sys
sys.path.append('..')

import torch
import torch.nn as nn
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import pickle

## 1. Load Model & Data

In [14]:
# Load preprocessed data and model
import json
import sentencepiece as spm
from src.model import Transformer
from src.utils import get_device



# Load tokenizer config
with open('../data/processed/tokenizer_info.json', 'r') as f:
    tokenizer_info = json.load(f)

# Load SentencePiece tokenizers
sp_vi = spm.SentencePieceProcessor()
sp_vi.load(tokenizer_info['vi_model'])
sp_en = spm.SentencePieceProcessor()
sp_en.load(tokenizer_info['en_model'])

# Special token IDs
pad_id = tokenizer_info['pad_id']
bos_id = tokenizer_info['bos_id']
eos_id = tokenizer_info['eos_id']

# Load processed splits
with open('../data/processed/splits.pkl', 'rb') as f:
    splits = pickle.load(f)
train_data = splits['train']
val_data = splits['val']
test_data = splits['test']

# Dataset class (reuse from preprocessing)
from torch.utils.data import Dataset, DataLoader
class TranslationDataset(Dataset):
    def __init__(self, data, sp_src, sp_tgt, max_length=128):
        self.data = data
        self.sp_src = sp_src
        self.sp_tgt = sp_tgt
        self.max_length = max_length
        self.pad_id = pad_id
        self.bos_id = bos_id
        self.eos_id = eos_id
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        item = self.data[idx]
        src_ids = self.sp_src.encode_as_ids(item['vi'])
        src_ids = [self.bos_id] + src_ids + [self.eos_id]
        tgt_ids = self.sp_tgt.encode_as_ids(item['en'])
        tgt_ids = [self.bos_id] + tgt_ids + [self.eos_id]
        if len(src_ids) > self.max_length:
            src_ids = src_ids[:self.max_length-1] + [self.eos_id]
        if len(tgt_ids) > self.max_length:
            tgt_ids = tgt_ids[:self.max_length-1] + [self.eos_id]
        return {
            'src': torch.tensor(src_ids, dtype=torch.long),
            'tgt': torch.tensor(tgt_ids, dtype=torch.long),
            'src_text': item['vi'],
            'tgt_text': item['en']
        }

def collate_fn(batch):
    src_batch = [item['src'] for item in batch]
    tgt_batch = [item['tgt'] for item in batch]
    src_padded = nn.utils.rnn.pad_sequence(src_batch, batch_first=True, padding_value=pad_id)
    tgt_padded = nn.utils.rnn.pad_sequence(tgt_batch, batch_first=True, padding_value=pad_id)
    return {
        'src': src_padded,
        'tgt': tgt_padded
    }

# Create DataLoaders


## 2. Training Configuration

In [15]:
# Hyperparameters
config = {
    'd_model': 512,
    'num_heads': 8,
    'num_encoder_layers': 6,
    'num_decoder_layers': 6,
    'd_ff': 2048,
    'max_len': tokenizer_info['max_length'],
    'dropout': 0.1,
    'batch_size': 8,
    'num_epochs': 5,  # thử 5 epoch trước
    'learning_rate': 1e-4,  # Adam lr, NoamScheduler sẽ điều chỉnh lại
    'warmup_steps': 4000,
    'label_smoothing': 0.1,
    'checkpoint_dir': '../checkpoints',
    'log_dir': '../logs',
}

os.makedirs(config['checkpoint_dir'], exist_ok=True)
os.makedirs(config['log_dir'], exist_ok=True)

batch_size = config['batch_size']
train_dataset = TranslationDataset(train_data, sp_vi, sp_en, max_length=tokenizer_info['max_length'])
val_dataset = TranslationDataset(val_data, sp_vi, sp_en, max_length=tokenizer_info['max_length'])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=True)

# Initialize model
model = Transformer(
    src_vocab_size=tokenizer_info['vi_vocab_size'],
    tgt_vocab_size=tokenizer_info['en_vocab_size'],
    d_model=512,
    num_heads=8,
    num_encoder_layers=6,
    num_decoder_layers=6,
    d_ff=2048,
    max_len=tokenizer_info['max_length'],
    dropout=0.1,
    pad_idx=pad_id
)

device = get_device()
model = model.to(device)
print(f"Model loaded on {device}")
print(f"Train samples: {len(train_dataset)} | Val samples: {len(val_dataset)}")

Using GPU: NVIDIA GeForce RTX 4060 Laptop GPU
GPU Memory: 8.59 GB
Model loaded on cuda
Train samples: 300000 | Val samples: 25000


## 3. Loss Function & Optimizer

In [16]:
# Setup loss function
from src.utils import LabelSmoothingLoss

criterion = LabelSmoothingLoss(
    vocab_size=tokenizer_info['en_vocab_size'],
    padding_idx=pad_id,
    smoothing=config['label_smoothing']
)

# Setup optimizer
optimizer = Adam(model.parameters(), lr=config['learning_rate'], betas=(0.9, 0.98), eps=1e-9)

## 4. Learning Rate Scheduler

In [17]:
from src.utils import NoamScheduler

def get_lr_scheduler(optimizer, d_model, warmup_steps):
    return NoamScheduler(optimizer, d_model, warmup_steps)

scheduler = get_lr_scheduler(optimizer, config['d_model'], config['warmup_steps'])

## 5. Training Functions

In [18]:
def train_epoch(model, pbar, optimizer, criterion, scheduler, device, checkpoint_dir, grad_accum_steps=1, pad_id=0, epoch=1, resume_step=0):
    model.train()
    running_loss = 0.0
    total_tokens = 0
    optimizer.zero_grad()
    
    current_ckpt_path = os.path.join(checkpoint_dir, 'current_checkpoint.pt')
    
    # Biến đếm số bước thực tế đã train trong epoch này (để chia trung bình)
    step_count = 0 
    
    for i, batch in enumerate(pbar, 1):
        # 1. Logic Skip (Nhanh, nhưng vẫn tốn thời gian load data từ ổ cứng nếu resume sâu)
        if i <= resume_step:
            continue
        
        src = batch['src'].to(device)
        tgt = batch['tgt'].to(device)
        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:].contiguous().view(-1)
        
        output = model(src, tgt_input)
        output = output.contiguous().view(-1, output.size(-1))
        
        loss = criterion(output, tgt_output) / grad_accum_steps
        loss.backward()
        
        if i % grad_accum_steps == 0 or i == len(pbar):
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        # Chỉ gọi .item() 1 lần duy nhất cho loss (Giảm sync GPU-CPU)
        # Loss này là Mean Loss của batch (trên valid token)
        current_loss = loss.item() * grad_accum_steps
        # Đếm số token hợp lệ (không phải padding)
        n_tokens = (tgt_output != pad_id).sum().item()
        running_loss += current_loss * n_tokens
        total_tokens += n_tokens
        step_count += 1
        
        # Hiển thị
        pbar.set_postfix({'loss': f'{current_loss:.4f}'})
        
        # Lưu checkpoint (Dùng current_loss để lưu, nhanh gọn)
        if i % 5000 == 0: # Lưu ý: Để 1000 hoặc 5000 thôi, 20000 hơi lâu
            save_checkpoint(model, optimizer, epoch, current_loss, current_ckpt_path, step=i)
    # Trả về trung bình cộng loss trên số token hợp lệ
    return running_loss / total_tokens if total_tokens > 0 else 0


def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    total_tokens = 0
    step_count = 0
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc='Val', leave=False):
            src = batch['src'].to(device)
            tgt = batch['tgt'].to(device)
            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:].contiguous().view(-1)
            output = model(src, tgt_input)
            output = output.contiguous().view(-1, output.size(-1))
            loss = criterion(output, tgt_output)
            n_tokens = (tgt_output != pad_id).sum().item()
            running_loss += loss.item() * n_tokens
            total_tokens += n_tokens
            step_count += 1
    return running_loss / total_tokens if total_tokens > 0 else 0

## 6. Training Loop

In [19]:
import os
import torch
import time
from tqdm import tqdm
from src.utils import save_checkpoint, calculate_perplexity

# --- PHẦN 1: LOGIC KHỞI TẠO & RESUME ---
resume_path = os.path.join(config['checkpoint_dir'], 'current_checkpoint.pt')

# Giá trị mặc định (Train từ đầu)
start_epoch = 1
start_step = 0
best_val_loss = float('inf')

# Kiểm tra nếu có file current -> Load để chạy tiếp
if os.path.exists(resume_path):
    print(f"--> Phát hiện file resume: {resume_path}")
    checkpoint = torch.load(resume_path)
    
    # 1. Load trọng số Model & Optimizer
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    # 2. Cập nhật vị trí epoch và step đang chạy dở
    start_epoch = checkpoint['epoch']
    start_step = checkpoint.get('step', 0)
    
    # (Tuỳ chọn) Load lại loss cũ nếu có lưu để so sánh best model
    if 'loss' in checkpoint:
         print(f"--> Loss lần cuối: {checkpoint['loss']:.4f}")

    print(f"--> Hệ thống sẽ RESUME từ Epoch {start_epoch}, Step {start_step}")
else:
    print("--> Không tìm thấy checkpoint. Bắt đầu train từ đầu (Epoch 1).")

train_losses, val_losses, val_ppls, lrs = [], [], [], []


# --- PHẦN 2: VÒNG LẶP CHÍNH (ĐÃ SỬA) ---

# Thay vì range(1, ...), ta dùng range(start_epoch, ...)
for epoch in range(start_epoch, config['num_epochs'] + 1):
    start_time = time.time()
    
    # Tính toán step cần bỏ qua (Skip):
    # - Nếu là epoch đang chạy dở (start_epoch) -> Skip 'start_step' bước.
    # - Nếu là epoch mới hoàn toàn -> Không skip (bằng 0).
    step_to_skip = start_step if epoch == start_epoch else 0
    
    # initial=step_to_skip: Giúp thanh tiến trình hiển thị đúng % ngay khi bắt đầu
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}", leave=False, initial=step_to_skip)
    
    # Gọi hàm train_epoch với tham số resume_step
    train_loss = train_epoch(model, pbar, optimizer, criterion, scheduler, device, 
                             checkpoint_dir=config['checkpoint_dir'], 
                             epoch=epoch,
                             resume_step=step_to_skip,
                             pad_id=pad_id) # <--- QUAN TRỌNG: Truyền bước cần skip vào và pad_id
    
    # QUAN TRỌNG: Sau khi chạy xong epoch dang dở, reset start_step về 0 
    # để các epoch sau chạy full từ đầu.
    start_step = 0 
    
    # --- PHẦN VALIDATE & SAVE (GIỮ NGUYÊN) ---
    val_loss = validate(model, val_loader, criterion, device)
    elapsed = time.time() - start_time
    perplexity = calculate_perplexity(val_loss)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_ppls.append(perplexity)
    lrs.append(optimizer.param_groups[0]['lr'])
    
    # Save checkpoint epoch (Lưu trữ lịch sử)
    ckpt_path = os.path.join(config['checkpoint_dir'], f'transformer_epoch_{epoch}.pt')
    save_checkpoint(model, optimizer, epoch, val_loss, ckpt_path)
    
    # Save best model (Lưu model tốt nhất)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_ckpt_path = os.path.join(config['checkpoint_dir'], 'best_transformer.pt')
        save_checkpoint(model, optimizer, epoch, val_loss, best_ckpt_path)
    
    tqdm.write(f"Epoch {epoch:2d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val PPL: {perplexity:.2f} | LR: {lrs[-1]:.6f} | Time: {elapsed:.1f}s | Ckpt: {ckpt_path}")

--> Không tìm thấy checkpoint. Bắt đầu train từ đầu (Epoch 1).


KeyboardInterrupt: 

## 7. Visualization

In [ ]:
# TO DO: Plot training curves
# - Loss curves (train vs validation)
# - Perplexity
# - Learning rate schedule

In [ ]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA")

In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.enabled)

In [ ]:
%pip install torch --index-url https://download.pytorch.org/whl/cu121

## 8. Debug: Kiểm tra dữ liệu và logits đầu ra

In [ ]:
# Kiểm tra dữ liệu train/val: số lượng, độ dài, trùng lặp, rỗng, lỗi token hóa
print('Số mẫu train:', len(train_data))
print('Số mẫu val:', len(val_data))
src_lens = [len(sp_vi.encode_as_ids(item['vi'])) for item in train_data]
tgt_lens = [len(sp_en.encode_as_ids(item['en'])) for item in train_data]
print('Độ dài src trung bình:', sum(src_lens)/len(src_lens), '| max:', max(src_lens), '| min:', min(src_lens))
print('Độ dài tgt trung bình:', sum(tgt_lens)/len(tgt_lens), '| max:', max(tgt_lens), '| min:', min(tgt_lens))
print('Số câu src rỗng:', sum([len(item['vi'].strip())==0 for item in train_data]))
print('Số câu tgt rỗng:', sum([len(item['en'].strip())==0 for item in train_data]))
print('Số cặp trùng lặp:', len(train_data) - len(set((item['vi'], item['en']) for item in train_data)))
print('Một số mẫu train:')
for i in range(3):
    print(f"src: {train_data[i]['vi']}")
    print(f"tgt: {train_data[i]['en']}")
    print('---')

Số mẫu train: 300000
Số mẫu val: 25000
Độ dài src trung bình: 23.655416666666667 | max: 135 | min: 3
Độ dài tgt trung bình: 20.765993333333334 | max: 132 | min: 3
Số câu src rỗng: 0
Số câu tgt rỗng: 0
Số cặp trùng lặp: 1
Một số mẫu train:
src: Câu chuyện bắt đầu với buổi lễ đếm ngược .
tgt: It begins with a countdown .
---
src: Ngày 14 , tháng 8 , năm 1947 , gần nửa đêm , ở Bombay , có một phụ nữ sắp lâm bồn .
tgt: On August 14th , 1947 , a woman in Bombay goes into labor as the clock ticks towards midnight .
---
src: Cùng lúc , trên khắp đất Ấn , người ta nín thở chờ đợi tuyên ngôn độc lập sau gần hai thập kỷ là thuộc địa của Anh .
tgt: Across India , people hold their breath for the declaration of independence after nearly two centuries of British occupation and rule .
---


In [ ]:
# Diagnostic -> write full logs to ../logs/diag_decode_output.txt
import torch, torch.nn.functional as F, json
from pathlib import Path
from src.evaluate import load_tokenizers_and_config
from src.utils import get_device

tokenizer_info, sp_vi, sp_en = load_tokenizers_and_config()
device = get_device()
model.to(device)
model.eval()

def piece_for_id(sp, i):
    try:
        return sp.IdToPiece(i)
    except Exception:
        try:
            return sp.id_to_piece(i)
        except Exception:
            try:
                return sp.decode_ids([i])
            except Exception:
                return f"<id:{i}>"

logs = []
def log(s):
    logs.append(str(s))

def inspect_example_to_logs(src_text, max_len=60):
    src_ids = sp_vi.encode_as_ids(src_text)
    src_ids = [tokenizer_info['bos_id']] + src_ids + [tokenizer_info['eos_id']]
    src = torch.tensor(src_ids, dtype=torch.long, device=device).unsqueeze(0)
    with torch.no_grad():
        encoder_output, src_mask = model.encode(src)
    tgt_ids = [tokenizer_info['bos_id']]
    log(f"SRC: {src_text}")
    log(f"SRC ids: {src_ids}")
    for step in range(max_len):
        tgt = torch.tensor(tgt_ids, dtype=torch.long, device=device).unsqueeze(0)
        with torch.no_grad():
            out = model.decode(tgt, encoder_output, src_mask)
        logits = out[0, -1, :]
        probs = F.softmax(logits, dim=-1)
        topk = torch.topk(probs, k=10)
        next_id = int(torch.argmax(logits).item())
        log(f"Step {step+1}: next_id={next_id} piece={piece_for_id(sp_en, next_id)} prob={probs[next_id].item():.6f}")
        log('  Top10: ' + json.dumps([(int(idx), piece_for_id(sp_en,int(idx)), float(p)) for idx,p in zip(topk.indices.tolist(), topk.values.tolist())]))
        ent = -(probs * torch.log(probs + 1e-12)).sum().item()
        log(f"  Entropy: {ent:.6f}")
        tgt_ids.append(next_id)
        if next_id == tokenizer_info['eos_id']:
            log('  EOS reached.')
            break
    log(f"Decoded ids: {tgt_ids}")
    decoded = [i for i in tgt_ids[1:] if i != tokenizer_info['eos_id'] and i != tokenizer_info['pad_id']]
    try:
        log('Decoded text (robust): ' + sp_en.decode_ids(decoded))
    except Exception:
        log('Decoded text (fallback): ' + ''.join([piece_for_id(sp_en,i) for i in decoded]).replace('▁',' ').strip())
    log('----')

examples = [
    "Lý tưởng nhất là bạn bắt đầu ngày mới với 10-15 phút thiền , hoặc áp dụng các kỹ thuật thư giãn tinh thần với bài tập hít thở .",
    "Chỉ cần vài phút ngồi thiền cũng đủ đảm bảo một khởi đầu tích cực trong ngày , giúp bạn đối phó tốt hơn với sự căng thẳng .",
    "Giảm căng thẳng cũng đồng nghĩa giảm tình trạng viêm gây tắc các ống dẫn trong cơ thể ."
]
for ex in examples:
    inspect_example_to_logs(ex)

out_path = Path('..') / 'logs' / 'diag_decode_output.txt'
out_path.parent.mkdir(parents=True, exist_ok=True)
with open(out_path, 'w', encoding='utf-8') as f:
    for line in logs:
        f.write(line + '\n')
print(f'Wrote diagnostic logs to: {out_path.resolve()}')

SyntaxError: unterminated string literal (detected at line 71) (2732133536.py, line 71)

In [ ]:
# Lấy 1 batch từ train_loader để kiểm tra
batch = next(iter(train_loader))
src = batch['src']
tgt = batch['tgt']
print('src shape:', src.shape)
print('tgt shape:', tgt.shape)
print('src[0]:', src[0].tolist())
print('tgt[0]:', tgt[0].tolist())
print('src_text:', train_dataset.sp_src.decode_ids([i for i in src[0].tolist() if i != pad_id]))
print('tgt_text:', train_dataset.sp_tgt.decode_ids([i for i in tgt[0].tolist() if i != pad_id]))

src shape: torch.Size([8, 63])
tgt shape: torch.Size([8, 58])
src[0]: [2, 116, 388, 497, 1111, 6281, 36, 385, 495, 680, 510, 82, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tgt[0]: [2, 95, 12, 13842, 5530, 376, 324, 93, 1294, 4715, 15, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
src_text: Và hệ thống chữ Indus có tính chất đặc biệt này
tgt_text: And the Indus script now has this particular property .


In [ ]:
# Đưa batch lên device, tạo input cho model
src = src.to(device)
tgt = tgt.to(device)
tgt_input = tgt[:, :-1]
tgt_output = tgt[:, 1:]
with torch.no_grad():
    logits = model(src, tgt_input)  # [B, T, vocab_size]
print('logits shape:', logits.shape)
print('logits[0,0,:10]:', logits[0,0,:10].cpu().numpy())
print('logits min:', logits.min().item(), '| max:', logits.max().item(), '| mean:', logits.mean().item())

logits shape: torch.Size([8, 57, 32000])
logits[0,0,:10]: [-0.5386893  -0.7801632  -0.32187152  2.3923707  -0.95442057 -0.60122186
 -0.16280548  1.167485   -0.1504313   2.2502346 ]
logits min: -1.6018061637878418 | max: 2.59318208694458 | mean: -0.6912190318107605


In [ ]:
# Tính loss trên batch này, so sánh với cross-entropy không smoothing
from torch.nn import CrossEntropyLoss
logits_flat = logits.contiguous().view(-1, logits.size(-1))
tgt_output_flat = tgt_output.contiguous().view(-1)
loss_label_smooth = criterion(logits_flat, tgt_output_flat).item()
ce_loss = CrossEntropyLoss(ignore_index=pad_id)
loss_ce = ce_loss(logits_flat, tgt_output_flat).item()
print(f"LabelSmoothingLoss: {loss_label_smooth:.4f} | CrossEntropyLoss: {loss_ce:.4f}")

LabelSmoothingLoss: 3.7305 | CrossEntropyLoss: 9.1884


In [ ]:
# Kiểm tra phân phối xác suất sau softmax (có bị collapse không)
import torch.nn.functional as F
probs = F.softmax(logits, dim=-1)
print('probs[0,0,:10]:', probs[0,0,:10].cpu().numpy())
print('probs[0,0] sum:', probs[0,0].sum().item())
print('probs max:', probs.max().item(), '| min:', probs.min().item(), '| mean:', probs.mean().item())

probs[0,0,:10]: [3.4578192e-05 2.7160109e-05 4.2950192e-05 6.4825447e-04 2.2816685e-05
 3.2482152e-05 5.0355458e-05 1.9045149e-04 5.0982435e-05 5.6236284e-04]
probs[0,0] sum: 1.0
probs max: 0.0007970688166096807 | min: 1.2010872524115257e-05 | mean: 3.125000148429535e-05
